# Experiments

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""  # API key is sensitive to share
os.environ["LANGSMITH_API_KEY"] = ""   # API key is sensitive to share
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

In [3]:
pip install langchain langchain-community langchain-openai scikit-learn openai nest_asyncio

Here is the RAG Application that we've been working with throughout this course

In [4]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


Fetching pages: 100%|##########| 197/197 [00:10<00:00, 18.10it/s]


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [5]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-e46536a8' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=2368f543-fa44-4833-9910-8d01928d4dc5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.930399,1868b359-b3ee-4908-8655-ae9f0184bc56,85a8edcd-b65d-452d-9d51-5d78c64a0916
1,Does LangSmith support offline evaluation?,The provided context does not mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.516065,5524be6a-278d-4d44-830c-49aa9a66fe4d,fba3cc72-e1ef-46b6-9b2e-f8da096b3e6d
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.873669,b317d976-030d-4bdc-ad85-71bde482f46c,236b6e9a-ea35-4518-8953-82831a4e65c5
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.395413,ca92d9ca-b757-4ec9-b317-33c17f60046f,4f7bbdeb-7321-4893-9ff9-eefa1f9c795d
4,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript/TypeScript SDK for ...",None,"Yes, there is a Javascript Langsmith SDK!",1,1.028646,cc388aab-a574-48d3-b1ca-050b4e55eca4,a6e6a1c8-8b85-4f88-b0ee-f44c7f4a2190
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.452049,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,83046838-8a57-4c1c-ae90-0c3b7ef4842c
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.784284,b4fd0e07-e10c-4462-bc59-1b1b24e76445,ef5bbe42-d98c-4143-a21a-cfdce6e88460
7,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.963302,fb838522-26d9-4966-b272-b687b41c4df7,a730d425-1ebe-4031-a222-e7f0b892e8e1
8,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.401892,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,908133d1-0437-4e44-83e3-47c755da6ef7
9,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,0,3.726944,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,d668d359-5753-450a-9698-e78d696070dd


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [6]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-4e6045c6' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=80f3521b-78da-4275-aecb-c21b492687ad




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.294404,1868b359-b3ee-4908-8655-ae9f0184bc56,fa4d7c91-05d9-4ef3-8ad8-375ba923ee2a
1,Does LangSmith support offline evaluation?,"No, LangSmith primarily supports online evalua...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.105574,5524be6a-278d-4d44-830c-49aa9a66fe4d,e53b949f-945c-4e89-8219-c81f1448a405
2,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.941372,b317d976-030d-4bdc-ad85-71bde482f46c,9beb4c35-0791-44ee-8547-7ba053f9b33e
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.314620,ca92d9ca-b757-4ec9-b317-33c17f60046f,08ec4e18-64b5-4352-b590-471e7cb1d71a
4,Is there a Javascript Langsmith SDK,"Yes, there is a Javascript/TypeScript SDK for ...",None,"Yes, there is a Javascript Langsmith SDK!",1,0.562758,cc388aab-a574-48d3-b1ca-050b4e55eca4,04275f78-cbbf-46f4-b7a5-bf06e024e488
5,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.698199,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,c6e66cd4-b931-425e-8857-17ed94c0edce
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation, off...",None,"Yes, LangSmith supports online evaluation as a...",1,1.409910,b4fd0e07-e10c-4462-bc59-1b1b24e76445,2b41d695-c871-4352-9410-849827633de4
7,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.132145,fb838522-26d9-4966-b272-b687b41c4df7,25ba0e39-5187-486b-978b-81bf3a015798
8,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,1,2.189491,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,6ccc5c06-47df-451a-9685-60a9f2f369ac
9,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you includ...",None,You can pass metadata with the @traceable deco...,1,1.522167,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,e39b9d93-c43f-4d62-a74e-6228f4bf5f55


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [7]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-006c870d' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=19826740-f1a2-44ff-8d02-631c1e2dbd4c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,5.638451,3392f6ef-a421-4bee-b347-7d5556480698,fe395aff-48ce-4388-8118-c620152250c9
1,How can I trace with the @traceable decorator?,To trace with the `@traceable` decorator in Py...,None,To trace with the @traceable decorator in Pyth...,1,2.853303,5b2929e1-6b02-4bd5-aada-fc208a08dc28,f3ffbc3a-64dc-488d-aa5d-008481dce9e4
2,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.717876,7f031e8d-2536-42b0-9381-827d46b7f048,e79bbfe6-05e1-49d1-886a-6c981a30bc33
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on LLM observab...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.882249,80ef51fb-9c02-4e83-9b33-34603867ea0a,56a1a0f1-2ddc-445b-8352-605a68cd7be8
4,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,0,2.403239,8b49c7b1-1535-45f0-b87f-fc2d3879b681,b67c5391-7002-4b3d-a195-102973fcdf94
5,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,2.307009,915513f0-5215-46c4-9ba2-5f063706bee2,4ab33737-8c8f-40f6-8181-149e850c4e41
6,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,2.234782,9ac0937d-6b53-4656-a9d9-ac3b25ee59b0,dcf37bfa-249f-4c53-8e10-daf86b514c4a
7,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.544620,9acc4f8c-7528-4b88-9b4a-683033a3a16f,7560f75b-34e4-41ab-8c8f-75c6f1b36353
8,Does LangSmith support offline evaluation?,"No, LangSmith primarily supports online evalua...",None,"Yes, LangSmith supports offline evaluation thr...",1,0.780606,c7dda75f-b182-4b11-920b-db6c12c63b39,46215fbb-a089-421d-a8df-13c8280687f4
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.433706,fa99e052-566d-498c-af32-0d5dfc91c148,695c74a4-6c3f-4153-a535-e8d9d87369d7


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [8]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-87af62bd' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=f160cb76-85b1-4ba1-b475-9b36b39e3a12




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,3.662065,1868b359-b3ee-4908-8655-ae9f0184bc56,1f721e4a-5ab6-4a5f-813e-0313f3651735
1,Does LangSmith support offline evaluation?,The provided context does not specifically men...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.227076,5524be6a-278d-4d44-830c-49aa9a66fe4d,7b924710-1f5a-46b9-bc0d-9bd100448713
2,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.662854,b317d976-030d-4bdc-ad85-71bde482f46c,1359998a-ad6d-47be-a07a-e9b19bf430fe
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.214066,ca92d9ca-b757-4ec9-b317-33c17f60046f,522ef123-ccfc-47b3-8f5b-8fafccbd14ff
4,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript/TypeScript (JS/TS) ...",None,"Yes, there is a Javascript Langsmith SDK!",0,0.675009,cc388aab-a574-48d3-b1ca-050b4e55eca4,26b92f49-f02d-4060-bb3f-3a516d016a51
5,What is LangSmith used for in three sentences?,LangSmith is a platform used for building prod...,None,LangSmith is a platform designed for the devel...,1,1.103858,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,53f65a28-fbf1-41a7-a196-34ef94de74d2
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.320787,b4fd0e07-e10c-4462-bc59-1b1b24e76445,e5f9a530-a529-4cc9-9663-355c83b3f3e3
7,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.367073,fb838522-26d9-4966-b272-b687b41c4df7,a29d8e2e-292f-492c-ba16-ebb688e96944


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [10]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name,
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "1868b359-b3ee-4908-8655-ae9f0184bc56",
            "ca92d9ca-b757-4ec9-b317-33c17f60046f",
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-b81be6d7' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=a5b709ce-3be5-4f1a-9d03-8e9562e2523c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator using t...,None,To trace with the @traceable decorator in Pyth...,1,2.344642,1868b359-b3ee-4908-8655-ae9f0184bc56,a08c66da-110b-40c2-982f-930eff4ed9e0
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.688666,ca92d9ca-b757-4ec9-b317-33c17f60046f,ca987532-a19d-4049-a12b-12f03197c1b4


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [11]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-afc3bf75' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=79bf250c-aa7e-42c0-887e-02a29225cf29




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.450081,1868b359-b3ee-4908-8655-ae9f0184bc56,5938265a-6025-4648-814d-07c1fe7a777f
1,Does LangSmith support offline evaluation?,The content provided doesn't explicitly mentio...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.656318,5524be6a-278d-4d44-830c-49aa9a66fe4d,4adf6bb6-9bd5-48de-9dfe-d2431faae224
2,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,2.899961,b317d976-030d-4bdc-ad85-71bde482f46c,5f203b6e-4b4c-41ce-8351-3cae5f9e0552
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.332536,ca92d9ca-b757-4ec9-b317-33c17f60046f,1aeaad0f-0cf4-42e6-81a0-68f4560f9df4
4,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript/TypeScript (JS/TS) ...",None,"Yes, there is a Javascript Langsmith SDK!",0,1.029401,cc388aab-a574-48d3-b1ca-050b4e55eca4,eb6bbe87-8c66-41f9-b2fe-3a8d5b8a5380
5,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,1.592829,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,474e1047-1df0-441e-ae55-c32572317fa0
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.304463,b4fd0e07-e10c-4462-bc59-1b1b24e76445,89fc4d1b-2ca9-4677-b848-75e97d00954b
7,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.209173,fb838522-26d9-4966-b272-b687b41c4df7,5e801b9e-6fab-431a-bec9-1ce3d77fbcd6
8,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.698386,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,9bcfb007-27d8-4937-8e3f-a68edb345c6a
9,How do I pass metadata in with @traceable?,"To pass metadata with @traceable in LangSmith,...",None,You can pass metadata with the @traceable deco...,1,1.738085,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,0fc36057-d2a5-4c3e-ad3e-98cbe350f6cd


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-3a0b74ff' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=4ac124e2-da35-4e31-a854-44c05f16bd07




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-dG527CqlOVsdPysVPpv4BJC5 on tokens per min (TPM): Limit 30000, Used 28759, Requested 1490. Please try again in 498ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/tmp/ipython-input-862480817.py", line 5, in target_function
    return langsmith_rag(inputs["question"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 111, in langsmith_rag
    response = generate_response(question, documents)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 83, in generate_res

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id,feedback.wrapper
0,Does LangSmith support offline evaluation?,The provided context does not mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1.0,1.636948,5524be6a-278d-4d44-830c-49aa9a66fe4d,829592a5-f95c-4268-b932-5b755a41fe30,NaN
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1.0,1.782347,b317d976-030d-4bdc-ad85-71bde482f46c,d94caeea-0677-4373-b09e-9f6f07484475,NaN
2,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1.0,2.210017,1868b359-b3ee-4908-8655-ae9f0184bc56,4c4c69bc-f12d-4771-b145-07d36345d8bd,NaN
3,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript/TypeScript SDK for ...",None,"Yes, there is a Javascript Langsmith SDK!",1.0,0.479570,cc388aab-a574-48d3-b1ca-050b4e55eca4,7725600c-0a88-4a2e-ad33-3c6bca54edb0,NaN
4,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1.0,1.617860,ca92d9ca-b757-4ec9-b317-33c17f60046f,ac1a4417-a28e-4530-9ecf-27e10662a2d2,NaN
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1.0,1.107151,b4fd0e07-e10c-4462-bc59-1b1b24e76445,ac167716-37b7-4bc4-a1a0-43d6a8273b89,NaN
6,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1.0,1.229571,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,5a0ac223-dda7-4f7c-a067-2b7a22b5031b,NaN
7,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1.0,1.470331,fb838522-26d9-4966-b272-b687b41c4df7,fc80c4cc-fff8-4ad2-a321-5065cb8400a3,NaN
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you includ...",None,You can pass metadata with the @traceable deco...,1.0,1.738592,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,c32ec249-f762-4b2b-8780-94aa69bf2496,NaN
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1.0,2.219697,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,a5123be4-d817-4502-a627-1712cf8f9101,NaN


##### Metadata

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-d6c0cfa4' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=93be385c-4353-4434-bbc5-21415ebe9506




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the `@traceable` decorator in Py...,None,To trace with the @traceable decorator in Pyth...,1,4.655576,1868b359-b3ee-4908-8655-ae9f0184bc56,4c464881-5e60-477b-bbbf-501172d6587b
1,Does LangSmith support offline evaluation?,The context provided does not explicitly menti...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.322109,5524be6a-278d-4d44-830c-49aa9a66fe4d,e3246662-131e-45a8-9e97-5cb763e1cf26
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.617545,b317d976-030d-4bdc-ad85-71bde482f46c,db9fef79-87cb-42f0-a552-a478898f52d2
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.391062,ca92d9ca-b757-4ec9-b317-33c17f60046f,7cdc086d-02cf-47cb-91ea-3e6ec3be43cc
4,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript SDK for LangSmith, ...",None,"Yes, there is a Javascript Langsmith SDK!",0,0.726392,cc388aab-a574-48d3-b1ca-050b4e55eca4,b2f04f2a-cc20-445a-9933-f612aee72f24
5,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,1.318762,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,14192f01-e12e-41fa-a580-8f2fd3d7840c
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,3.226238,b4fd0e07-e10c-4462-bc59-1b1b24e76445,c609ce7a-ec1e-467c-a41a-1a0f2d8800cb
7,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.225821,fb838522-26d9-4966-b272-b687b41c4df7,8a9e3f56-0afb-40a3-8098-2800115c3644
8,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,1.755523,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,f2c61ea9-915b-49ab-8d4f-c1a8bf143da2
9,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,4.110370,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,8aa519fb-e656-4b55-b17f-066531c49c41
